# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 

As of today, the easiest way to experiment with Ebiose is to use the OpenAI API. To do so, all you have to do is to set your OpenAI API key via an .env file or by replacing `"your-open-api-key"` in the following code block:

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

> 💡 Recall that you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so or execute the following:


In [2]:
import sys
sys.path.append("./../")


## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [3]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [4]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [5]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [6]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="gpt-4o-mini"
)

# Running a forge cycle

Once the forge is instantitated, we can start generating agents by running a **forge cycle**. 

We must first define a list of available LLMs. Here, we will use GPT 4o mini only. 

In [7]:

from ebiose.core.model_endpoint import ModelEndpoint, ModelSize, ModelType

available_models = [
    ModelEndpoint(
        endpoint_id="gpt-4o-mini",
        model_name="GPT-4o-mini",
    ),
]

In Ebiose, all calls to LLMs (and, in a next future, for any other type of call that costs) are managed by the `ComputeIntensiveBatchProcessor` static class. This class must be initialized with the list of available models, as follows:

In [8]:
from ebiose.compute_intensive_batch_processor.compute_intensive_batch_processor import (
    ComputeIntensiveBatchProcessor,
)
ComputeIntensiveBatchProcessor.initialize(available_models)

The forge cycle is based on an evolutionary algorithm that requires two types of specialized agents: 
- **architect agents** which are agents that generate other agents from scratch;
- **genetic operator agents** which generate agents from one or several other existing agents.

For now, hand-made architect and crossover agents are provided in the `GraphUtils` class. We can load them as follows:

In [9]:
from ebiose.core.engines.graph_engine.utils import GraphUtils

architect_agent = GraphUtils.get_architect_agent(model_endpoint_id="gpt-4o-mini")
crossover_agent = GraphUtils.get_crossover_agent(model_endpoint_id="gpt-4o-mini")

These agents are required to instantiate an Ebiose ecosystem:

In [10]:
from ebiose.core.ecosystem import Ecosystem

ecosystem = Ecosystem(
    initial_architect_agents=[architect_agent],
    initial_genetic_operator_agents=[crossover_agent],
)

The forge cycle can now be launched by providing the created ecosystem, a budget in dollars (the forge cycle will end once this budget is exhausted) and, optionally, a path in which created agents and fitness will be saved accross generations. Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [11]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)

# budget for the forge cycle, in dollars
BUDGET = 0.01

cycle_config = EvoForgingCylceConfig(
    budget=BUDGET,
    n_agents_in_population=2,
    n_selected_agents_from_ecosystem=0,
    replacement_ratio=0.5,
    save_path=SAVE_PATH
)

final_agents, final_fitness = asyncio.run(forge.run_new_cycle(ecosystem, cycle_config=cycle_config))


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
100%|██████████| 2/2 [00:33<00:00, 16.56s/it]
Agent initialization cost: 0.005591549999999999
Population initialized with 2 agents
Initialization of 2 agents took 0:00:33.154832
Budget left after initialization: 0.004408450000000001 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 6246.17it/s]
Agent agent-d7078ffa-3cf8-41e0-9e60-ba6810ac47f1 fitness: 0.29976699686368236, cost: 0.0
Agent agent-bdb5b2d1-9aa9-4e62-8c1c-37fc2f071c1a fitness: 0.7943794815224912, cost: 0.0
Evaluation took 0:00:00.002959 for a total cost of 0.0 $


# Agent ID: agent-bdb5b2d1-9aa9-4e62-8c1c-37fc2f071c1a
## Fitness: 0.7943794815224912
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) -->|understood| Llm_Node_2(Solution Generation LLM)
	Llm_Node_1(Problem Understanding LLM) -->|not understood| End_Node[end_node]
	Llm_Node_2(Solution Generation LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. This model consists of multiple nodes, each with a specific function in the problem-solving process. Your role is to contribute effectively to this process by understanding and processing the information provided by other nodes.

1. **Problem Understanding LLM**: Your task is to parse the math word problem, identify key components such as numbers, operations, and relationships, and formulate a clear understanding of what is being asked.

2. **Solution Generation LLM**: Your task is to take the clear understanding provided by the Problem Understanding LLM and generate a step-by-step solution to the math word problem. Apply relevant mathematical reasoning and operations to arrive at the solution.

All outputs should be detailed and clear to ensure smooth transitions between nodes. Use chain-of-thought reasoning and self-reflection where applicable to enhance the quality of your responses.
##### Problem Understanding LLM
You are tasked with analyzing the following math word problem. Your goal is to extract key components such as numbers, operations, and relationships, and articulate a clear understanding of what the problem is asking.

Please respond with the following:
1. A summary of the problem's context.
2. A list of key components identified, including numerical values and operations to be performed.
3. A clear statement of what the problem is asking for, formulated in your own words.

Make sure to reflect on the content, verifying that you have captured all necessary elements for the next step in the process. If you encounter any ambiguities or uncertainties in the problem, note them for clarification.
##### Solution Generation LLM
You are tasked with generating a solution to the math word problem based on the understanding provided by the Problem Understanding LLM. Use the key components identified and the clear statement of the problem to formulate a step-by-step solution.

Please respond with the following:
1. A detailed breakdown of the mathematical operations required to solve the problem.
2. The final answer, clearly stated, along with any relevant explanations or justifications for the steps taken.
3. If you identify any potential errors or areas needing reevaluation in the previous understanding, include constructive feedback for improvement.

Utilize chain-of-thought reasoning to guide your calculations, ensuring clarity and logic in each step of the solution process.

# Agent ID: agent-d7078ffa-3cf8-41e0-9e60-ba6810ac47f1
## Fitness: 0.29976699686368236
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Analyzer(MathProblemAnalyzer)
	Math_Problem_Analyzer(MathProblemAnalyzer) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a structured network of interconnected nodes. Your role is to analyze the given math problem, extract the necessary information, and prepare it for the next stage of the problem-solving process. You should leverage chain-of-thought reasoning to break down the problem, identify key components such as numbers, operations, and relationships, and provide a clear, concise summary of your findings. Your output will directly influence the subsequent steps taken in this problem-solving process.
##### MathProblemAnalyzer
Given the following math word problem, please analyze it in detail and extract key information necessary for solving it. Break down the problem step by step using chain-of-thought reasoning. Identify and list the numbers involved, the mathematical operations required (e.g., addition, subtraction, multiplication, division), and the relationships between the elements of the problem. 

For your analysis, consider the following steps:
1. Read the problem carefully.
2. Identify and extract the key numbers.
3. Determine what operations are needed to solve the problem.
4. Summarize the relationships you have found in a clear format.

Once you have processed this information, prepare a structured output that includes:
- A list of identified numbers
- A list of identified operations
- A brief explanation of the relationships you have found

Your output will serve as the foundational information for the next node in the graph, so clarity and precision are essential.



Saving current state to ../data/2025-03-03_20-48-42/generation=0
Starting crossover and mutation...
Initializing structured output agent for model with fields {'edges': FieldInfo(annotation=list[Edge], required=False, default_factory=list, description='list of edges in the graph'), 'nodes': FieldInfo(annotation=list[Union[StartNode, EndNode, LLMNode, PydanticValidatorNode, RegexRoutingNode]], required=False, default_factory=list, description='list of nodes in the graph'), 'description': FieldInfo(annotation=str, required=False, default=''), 'shared_context_prompt': FieldInfo(annotation=str, required=True, metadata=[MinLen(min_length=1)])} (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:27.806613 for a total cost of 0.002163 $
Generation 0 completed in 0:00:27.820863 with a total cost of 0.002163 $
Budget left after first generation: 0.0022454500000000013 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<

# Agent ID: agent-3578626c-00a8-4e50-93fb-1566b7af40de
## Fitness: 0.6682158565343952
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) -->|understood| Llm_Node_2(Solution Generation LLM)
	Llm_Node_1(Problem Understanding LLM) -->|not understood| Llm_Node_3(Clarification LLM)
	Llm_Node_2(Solution Generation LLM) --> End_Node[end_node]
	Llm_Node_3(Clarification LLM) -->|clarified| Llm_Node_2(Solution Generation LLM)
	Llm_Node_3(Clarification LLM) -->|still unclear| End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. This model consists of multiple nodes, each with a specific function in the problem-solving process. Your role is to contribute effectively to this process by understanding and processing the information provided by other nodes.

1. **Problem Understanding LLM**: Your task is to parse the math word problem, identify key components such as numbers, operations, and relationships, and formulate a clear understanding of what is being asked.

2. **Solution Generation LLM**: Your task is to take the clear understanding provided by the Problem Understanding LLM and generate a step-by-step solution to the math word problem. Apply relevant mathematical reasoning and operations to arrive at the solution.

3. **Clarification LLM**: This node is to be activated when the Problem Understanding LLM identifies uncertainties. It will seek to clarify any ambiguities before generating a solution.

All outputs should be detailed and clear to ensure smooth transitions between nodes. Use chain-of-thought reasoning and self-reflection where applicable to enhance the quality of your responses.
##### Problem Understanding LLM
You are tasked with analyzing the following math word problem. Your goal is to extract key components such as numbers, operations, and relationships, and articulate a clear understanding of what the problem is asking.

Please respond with the following:
1. A summary of the problem's context.
2. A list of key components identified, including numerical values and operations to be performed.
3. A clear statement of what the problem is asking for, formulated in your own words.

Make sure to reflect on the content, verifying that you have captured all necessary elements for the next step in the process. If you encounter any ambiguities or uncertainties in the problem, note them for clarification.
##### Solution Generation LLM
You are tasked with generating a solution to the math word problem based on the understanding provided by the Problem Understanding LLM. Use the key components identified and the clear statement of the problem to formulate a step-by-step solution.

Please respond with the following:
1. A detailed breakdown of the mathematical operations required to solve the problem.
2. The final answer, clearly stated, along with any relevant explanations or justifications for the steps taken.
3. If you identify any potential errors or areas needing reevaluation in the previous understanding, include constructive feedback for improvement.

Utilize chain-of-thought reasoning to guide your calculations, ensuring clarity and logic in each step of the solution process.
##### Clarification LLM
In situations where the problem understanding is unclear, please provide additional context or clarification necessary to resolve ambiguities. Outline any potential misunderstandings or gaps in the initial interpretation, and request the user to provide further details if needed.

# Agent ID: agent-bdb5b2d1-9aa9-4e62-8c1c-37fc2f071c1a
## Fitness: 0.03920725704743766
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) -->|understood| Llm_Node_2(Solution Generation LLM)
	Llm_Node_1(Problem Understanding LLM) -->|not understood| End_Node[end_node]
	Llm_Node_2(Solution Generation LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. This model consists of multiple nodes, each with a specific function in the problem-solving process. Your role is to contribute effectively to this process by understanding and processing the information provided by other nodes.

1. **Problem Understanding LLM**: Your task is to parse the math word problem, identify key components such as numbers, operations, and relationships, and formulate a clear understanding of what is being asked.

2. **Solution Generation LLM**: Your task is to take the clear understanding provided by the Problem Understanding LLM and generate a step-by-step solution to the math word problem. Apply relevant mathematical reasoning and operations to arrive at the solution.

All outputs should be detailed and clear to ensure smooth transitions between nodes. Use chain-of-thought reasoning and self-reflection where applicable to enhance the quality of your responses.
##### Problem Understanding LLM
You are tasked with analyzing the following math word problem. Your goal is to extract key components such as numbers, operations, and relationships, and articulate a clear understanding of what the problem is asking.

Please respond with the following:
1. A summary of the problem's context.
2. A list of key components identified, including numerical values and operations to be performed.
3. A clear statement of what the problem is asking for, formulated in your own words.

Make sure to reflect on the content, verifying that you have captured all necessary elements for the next step in the process. If you encounter any ambiguities or uncertainties in the problem, note them for clarification.
##### Solution Generation LLM
You are tasked with generating a solution to the math word problem based on the understanding provided by the Problem Understanding LLM. Use the key components identified and the clear statement of the problem to formulate a step-by-step solution.

Please respond with the following:
1. A detailed breakdown of the mathematical operations required to solve the problem.
2. The final answer, clearly stated, along with any relevant explanations or justifications for the steps taken.
3. If you identify any potential errors or areas needing reevaluation in the previous understanding, include constructive feedback for improvement.

Utilize chain-of-thought reasoning to guide your calculations, ensuring clarity and logic in each step of the solution process.



Saving current state to ../data/2025-03-03_20-48-42/generation=1
Starting crossover and mutation...
BudgetExceededError: Master budget limit exceeded. Limit: 0.01, New total: 0.010207649999999999. Finishing process.
Error when calling gpt-4o-mini: Master budget limit exceeded. Limit: 0.01, New total: 0.010207649999999999
BudgetExceededError: Master budget limit exceeded. Limit: 0.01, New total: 0.011332199999999999. Finishing process.
Error when calling gpt-4o-mini: Master budget limit exceeded. Limit: 0.01, New total: 0.011332199999999999
BudgetExceededError: Master budget limit exceeded. Limit: 0.01, New total: 0.012456449999999999. Finishing process.
Error when calling gpt-4o-mini: Master budget limit exceeded. Limit: 0.01, New total: 0.012456449999999999
BudgetExceededError: Master budget limit exceeded. Limit: 0.01, New total: 0.01358595. Finishing process.
Error when calling gpt-4o-mini: Master budget limit exceeded. Limit: 0.01, New total: 0.01358595
Error while running agent ag

We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

In [12]:
forge.display_results(final_agents, final_fitness)

# Agent ID: agent-3578626c-00a8-4e50-93fb-1566b7af40de
## Fitness: 0.02256292805558857
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) -->|understood| Llm_Node_2(Solution Generation LLM)
	Llm_Node_1(Problem Understanding LLM) -->|not understood| Llm_Node_3(Clarification LLM)
	Llm_Node_2(Solution Generation LLM) --> End_Node[end_node]
	Llm_Node_3(Clarification LLM) -->|clarified| Llm_Node_2(Solution Generation LLM)
	Llm_Node_3(Clarification LLM) -->|still unclear| End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems. This model consists of multiple nodes, each with a specific function in the problem-solving process. Your role is to contribute effectively to this process by understanding and processing the information provided by other nodes.

1. **Problem Understanding LLM**: Your task is to parse the math word problem, identify key components such as numbers, operations, and relationships, and formulate a clear understanding of what is being asked.

2. **Solution Generation LLM**: Your task is to take the clear understanding provided by the Problem Understanding LLM and generate a step-by-step solution to the math word problem. Apply relevant mathematical reasoning and operations to arrive at the solution.

3. **Clarification LLM**: This node is to be activated when the Problem Understanding LLM identifies uncertainties. It will seek to clarify any ambiguities before generating a solution.

All outputs should be detailed and clear to ensure smooth transitions between nodes. Use chain-of-thought reasoning and self-reflection where applicable to enhance the quality of your responses.
##### Problem Understanding LLM
You are tasked with analyzing the following math word problem. Your goal is to extract key components such as numbers, operations, and relationships, and articulate a clear understanding of what the problem is asking.

Please respond with the following:
1. A summary of the problem's context.
2. A list of key components identified, including numerical values and operations to be performed.
3. A clear statement of what the problem is asking for, formulated in your own words.

Make sure to reflect on the content, verifying that you have captured all necessary elements for the next step in the process. If you encounter any ambiguities or uncertainties in the problem, note them for clarification.
##### Solution Generation LLM
You are tasked with generating a solution to the math word problem based on the understanding provided by the Problem Understanding LLM. Use the key components identified and the clear statement of the problem to formulate a step-by-step solution.

Please respond with the following:
1. A detailed breakdown of the mathematical operations required to solve the problem.
2. The final answer, clearly stated, along with any relevant explanations or justifications for the steps taken.
3. If you identify any potential errors or areas needing reevaluation in the previous understanding, include constructive feedback for improvement.

Utilize chain-of-thought reasoning to guide your calculations, ensuring clarity and logic in each step of the solution process.
##### Clarification LLM
In situations where the problem understanding is unclear, please provide additional context or clarification necessary to resolve ambiguities. Outline any potential misunderstandings or gaps in the initial interpretation, and request the user to provide further details if needed.

